<a href="https://colab.research.google.com/github/HemaGanesan243/RED-BUS/blob/main/REDBUS_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium

In [ ]:
links=[
       'https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile',#KADAMBA
       'https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile',#SOUTHBENGAL
       'https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile',#HRTC
       'https://www.redbus.in/online-booking/astc/?utm_source=rtchometile', #ASSAM
       'https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile',#WBTC
       'https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu',#CHANDIGARH
       'https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile',#PUNJAB
       'https://www.redbus.in/online-booking/north-bengal-state-transport-corporation',#NORTHBENGAL
       'https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile',#BIHAR
       'https://www.redbus.in/online-booking/kaac-transport']#KAAC



from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

######################################################NO.0 KADABA##########################################################################3
driver = webdriver.Chrome()
driver.get(links[0])
time.sleep(5)

def get_route_links(driver):
    route_links = []
    try:
        body = driver.find_elements(By.XPATH, "//a[@class='route']")
        route_links = [link.get_attribute('href') for link in body]
    except Exception as e:
        print(f"Error in getting route links: {e}")
    return route_links

urls_list = get_route_links(driver)
df = pd.DataFrame({'URLs': urls_list})

def navigate_and_collect_links(driver):
    all_links = []
    for page_number in range(1,5):  # Adjust the range based on the number of pages you want to navigate
        try:
            links_on_page = get_route_links(driver)
            all_links.extend(links_on_page)
            if page_number < 4:  # Only navigate if there are more pages
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page_number + 1}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
                time.sleep(1)
                print(f"Clicking on Page {page_number + 1}")
                next_page_button.click()
                WebDriverWait(driver, 10).until(EC.text_to_be_present_in_element(
                    (By.XPATH, '//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'),
                    str(page_number + 1)))
                print(f"Successfully navigated to page {page_number + 1}")
                time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break
    return all_links

all_links = navigate_and_collect_links(driver)


def click_view_buses(driver):
    b=driver.find_elements(By.CSS_SELECTOR,"div[class='button']")
    for i in range(len(b)-1,-1,-1):
        try:
            b[i].click()
        except Exception as e:
              print()

def scroll_down_page(driver):
    try:
        body = driver.find_element(By.TAG_NAME, "body")
        for _ in range(14):
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(2)
    except Exception as e:
        print(f"Error scrolling down the page: {e}")


def extract_bus_data(driver, urls):
    bus_data = []
    for url in urls:
        try:
            driver.get(url)
            time.sleep(5)  # Wait for the page to load
            click_view_buses(driver)
            scroll_down_page(driver)


            bus_names = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
            bus_types = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
            departures_time = driver.find_elements(By.XPATH, "//div[contains(@class, 'dp-time f-19 d-color f-bold')]")
            reachings_time = driver.find_elements(By.XPATH, "//div[contains(@class, 'bp-time f-19 d-color disp-Inline')]")
            durations = driver.find_elements(By.XPATH, "//div[contains(@class, 'dur l-color lh-24')]")
            seats_available = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
            prices = driver.find_elements(By.XPATH, "//div[contains(@class, 'fare d-block')]")
            star_ratings = driver.find_elements(By.XPATH, "//div[contains(@class, 'rating-sec lh-24')]")

            for i in range(len(bus_names)):
                bus_data.append({
                    "route_link": url,
                    "bus_name": bus_names[i].text if i < len(bus_names) else '',
                    "bus_type": bus_types[i].text if i < len(bus_types) else '',
                    "departure_time": departures_time[i].text if i < len(departures_time) else '',
                    "reaching_time": reachings_time[i].text if i < len(reachings_time) else'',
                    "duration": durations[i].text if i < len(durations) else '',
                    "seats_available": seats_available[i].text if i < len(seats_available) else '',
                    "price": prices[i].text if i < len(prices) else '',
                    "star_rating" : star_ratings[i].text if i < len(star_ratings) else ''
                })

        except Exception as e:
             print(f"Error extracting data for URL {url}: {e}")
    return bus_data


bus_data = extract_bus_data(driver, all_links)
for bus in bus_data:
    print(bus)

driver.quit()

bus_dataframe1 = pd.DataFrame(bus_data)
bus_dataframe1.to_csv('bus_dataframe1.csv', index=False)


#################################################################NO.1 SOUTHBENGAL#####################################################################TSRTC


driver = webdriver.Chrome()
driver.get(links[1])
time.sleep(5)

urls_list = get_route_links(driver)
df = pd.DataFrame({'URLs': urls_list})

def navigate_and_collect_links(driver):
    all_links = []
    for page_number in range(1,6):
        try:
            links_on_page = get_route_links(driver)
            all_links.extend(links_on_page)
            if page_number < 5:  # Only navigate if there are more pages
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page_number + 1}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
                time.sleep(1)
                print(f"Clicking on Page {page_number + 1}")
                next_page_button.click()
                WebDriverWait(driver, 10).until(EC.text_to_be_present_in_element(
                    (By.XPATH, '//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'),
                    str(page_number + 1)))
                print(f"Successfully navigated to page {page_number + 1}")
                time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break
    return all_links

all_links = navigate_and_collect_links(driver)
bus_data = extract_bus_data(driver, all_links)

for bus in bus_data:
    print(bus)

driver.quit()

bus_dataframe2 = pd.DataFrame(bus_data)
bus_dataframe2.to_csv('bus_dataframe2.csv', index=False)


#############################################################NO.2 HRTC#################################################################################
driver = webdriver.Chrome()
driver.get(links[2])
time.sleep(5)

urls_list = get_route_links(driver)
df = pd.DataFrame({'URLs': urls_list})

def navigate_and_collect_links(driver):
    all_links = []
    for page_number in range(1,6):
        try:
            links_on_page = get_route_links(driver)
            all_links.extend(links_on_page)
            if page_number < 5:  # Only navigate if there are more pages
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page_number + 1}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
                time.sleep(1)
                print(f"Clicking on Page {page_number + 1}")
                next_page_button.click()
                WebDriverWait(driver, 10).until(EC.text_to_be_present_in_element(
                    (By.XPATH, '//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'),
                    str(page_number + 1)))
                print(f"Successfully navigated to page {page_number + 1}")
                time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break
    return all_links

all_links = navigate_and_collect_links(driver)

bus_data = extract_bus_data(driver, all_links)

for bus in bus_data:
    print(bus)

bus_dataframe3 = pd.DataFrame(bus_data)
bus_dataframe3.to_csv('bus_dataframe3.csv', index=False)

#############################################################NO.3 ASSAM##########################################################################

driver = webdriver.Chrome()
driver.get(links[3])
time.sleep(5)

urls_list = get_route_links(driver)
df = pd.DataFrame({'URLs': urls_list})

def navigate_and_collect_links(driver):
    all_links = []
    for page_number in range(1,6):
        try:
            links_on_page = get_route_links(driver)
            all_links.extend(links_on_page)
            if page_number < 5:  # Only navigate if there are more pages
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page_number + 1}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
                time.sleep(1)
                print(f"Clicking on Page {page_number + 1}")
                next_page_button.click()
                WebDriverWait(driver, 10).until(EC.text_to_be_present_in_element(
                    (By.XPATH, '//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'),
                    str(page_number + 1)))
                print(f"Successfully navigated to page {page_number + 1}")
                time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break
    return all_links

all_links = navigate_and_collect_links(driver)
bus_data = extract_bus_data(driver, all_links)

for bus in bus_data:
    print(bus)

driver.quit()

bus_dataframe4 = pd.DataFrame(bus_data)
bus_dataframe4.to_csv('bus_dataframe4.csv', index=False)


##########################################################NO.4 WBTC############################################################################

driver = webdriver.Chrome()
driver.get(links[4])
time.sleep(5)

urls_list = get_route_links(driver)
df = pd.DataFrame({'URLs': urls_list})

def navigate_and_collect_links(driver):
    all_links = []
    for page_number in range(1,6):
        try:
            links_on_page = get_route_links(driver)
            all_links.extend(links_on_page)
            if page_number < 5:  # Only navigate if there are more pages
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page_number + 1}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
                time.sleep(1)
                print(f"Clicking on Page {page_number + 1}")
                next_page_button.click()
                WebDriverWait(driver, 10).until(EC.text_to_be_present_in_element(
                    (By.XPATH, '//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'),
                    str(page_number + 1)))
                print(f"Successfully navigated to page {page_number + 1}")
                time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break
    return all_links

all_links = navigate_and_collect_links(driver)

bus_data = extract_bus_data(driver, all_links)

for bus in bus_data:
    print(bus)

driver.quit()

bus_dataframe5 = pd.DataFrame(bus_data)
bus_dataframe5.to_csv('bus_dataframe5.csv', index=False)

#############################################################NO.5  CHANDIGARH#############################################################

driver = webdriver.Chrome()
driver.get(links[5])
time.sleep(5)

urls_list = get_route_links(driver)
df = pd.DataFrame({'URLs': urls_list})


def navigate_and_collect_links(driver):
    all_links = []
    for page_number in range(1,6):
        try:
            links_on_page = get_route_links(driver)
            all_links.extend(links_on_page)
            if page_number < 5:  # Only navigate if there are more pages
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page_number + 1}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
                time.sleep(1)
                print(f"Clicking on Page {page_number + 1}")
                next_page_button.click()
                WebDriverWait(driver, 10).until(EC.text_to_be_present_in_element(
                    (By.XPATH, '//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'),
                    str(page_number + 1)))
                print(f"Successfully navigated to page {page_number + 1}")
                time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break
    return all_links

all_links = navigate_and_collect_links(driver)
bus_data = extract_bus_data(driver, all_links)

for bus in bus_data:
    print(bus)

driver.quit()
bus_dataframe6 = pd.DataFrame(bus_data)
bus_dataframe6.to_csv('bus_dataframe6.csv', index=False)

#############################################################NO.6 PUNJAB#############################################################################

driver = webdriver.Chrome()
driver.get(links[6])
time.sleep(5)

urls_list = get_route_links(driver)
df = pd.DataFrame({'URLs': urls_list})


def navigate_and_collect_links(driver):
    all_links = []
    for page_number in range(1,4):
        try:
            links_on_page = get_route_links(driver)
            all_links.extend(links_on_page)
            if page_number < 3:  # Only navigate if there are more pages
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page_number + 1}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
                time.sleep(1)
                print(f"Clicking on Page {page_number + 1}")
                next_page_button.click()
                WebDriverWait(driver, 10).until(EC.text_to_be_present_in_element(
                    (By.XPATH, '//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'),
                    str(page_number + 1)))
                print(f"Successfully navigated to page {page_number + 1}")
                time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break
    return all_links

all_links = navigate_and_collect_links(driver)
bus_data = extract_bus_data(driver, all_links)

for bus in bus_data:
    print(bus)

bus_dataframe7 = pd.DataFrame(bus_data)
bus_dataframe7.to_csv('bus_dataframe7.csv', index=False)


#######################################################NO.7 NORTHBENGAL  ###########################################################################


driver = webdriver.Chrome()
driver.get(links[7])
time.sleep(5)

urls_list = get_route_links(driver)
df = pd.DataFrame({'URLs': urls_list})


def navigate_and_collect_links(driver):
    all_links = []
    for page_number in range(1,6):
        try:
            links_on_page = get_route_links(driver)
            all_links.extend(links_on_page)
            if page_number < 5:  # Only navigate if there are more pages
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page_number + 1}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
                time.sleep(1)
                print(f"Clicking on Page {page_number + 1}")
                next_page_button.click()
                WebDriverWait(driver, 10).until(EC.text_to_be_present_in_element(
                    (By.XPATH, '//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'),
                    str(page_number + 1)))
                print(f"Successfully navigated to page {page_number + 1}")
                time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break
    return all_links

all_links = navigate_and_collect_links(driver)
bus_data = extract_bus_data(driver, all_links)

for bus in bus_data:
    print(bus)

driver.quit()
bus_dataframe8 = pd.DataFrame(bus_data)
bus_dataframe8.to_csv('bus_dataframe8.csv', index=False)


#################################################################NO.8 BIHAR##########################################################3

driver = webdriver.Chrome()
driver.get(links[8])
time.sleep(5)

urls_list = get_route_links(driver)
df = pd.DataFrame({'URLs': urls_list})


def navigate_and_collect_links(driver):
    all_links = []
    for page_number in range(1,5):
        try:
            links_on_page = get_route_links(driver)
            all_links.extend(links_on_page)
            if page_number < 4:  # Only navigate if there are more pages
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page_number + 1}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
                time.sleep(1)
                print(f"Clicking on Page {page_number + 1}")
                next_page_button.click()
                WebDriverWait(driver, 10).until(EC.text_to_be_present_in_element(
                    (By.XPATH, '//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'),
                    str(page_number + 1)))
                print(f"Successfully navigated to page {page_number + 1}")
                time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break
    return all_links

all_links = navigate_and_collect_links(driver)
bus_data = extract_bus_data(driver, all_links)

for bus in bus_data:
    print(bus)
driver.quit()

bus_dataframe9 = pd.DataFrame(bus_data)
bus_dataframe9.to_csv('bus_dataframe9.csv', index=False)

############################################################NO.9 KAAC##################################################

driver = webdriver.Chrome()
driver.get(links[9])
time.sleep(5)

urls_list = get_route_links(driver)
df = pd.DataFrame({'URLs': urls_list})

def navigate_and_collect_links(driver):
    all_links = []
    for page_number in range(1,3):
        try:
            links_on_page = get_route_links(driver)
            all_links.extend(links_on_page)
            if page_number < 2:  # Only navigate if there are more pages
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page_number + 1}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
                time.sleep(1)
                print(f"Clicking on Page {page_number + 1}")
                next_page_button.click()
                WebDriverWait(driver, 10).until(EC.text_to_be_present_in_element(
                    (By.XPATH, '//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'),
                    str(page_number + 1)))
                print(f"Successfully navigated to page {page_number + 1}")
                time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break
    return all_links

all_links = navigate_and_collect_links(driver)
bus_data = extract_bus_data(driver, all_links)


for bus in bus_data:
    print(bus)

driver.quit()

bus_dataframe10 = pd.DataFrame(bus_data)
bus_dataframe10.to_csv('bus_dataframe10.csv', index=False)

all_data=pd.concat([bus_dataframe1, bus_dataframe2,bus_dataframe3,bus_dataframe4,bus_dataframe5,bus_dataframe6,bus_dataframe7,bus_dataframe8,bus_dataframe9,bus_dataframe10],axis=0)
print(all_data)

df=pd.DataFrame(all_data)
df.to_csv('FINAL.csv', index=False)


import pandas as pd
bus_data = pd.read_csv('FINAL1.csv')


import re

def extract_route_name(url):
    pattern = r'/bus-tickets/([a-zA-Z-]+)'
    match = re.search(pattern, url)
    if match:
        route_name = match.group(1)
        return route_name.title()
    return None


# Apply the function to the route_link column
bus_data['route_name'] = bus_data['route_link'].apply(extract_route_name)

# Print the resulting DataFrame
print(bus_data)

import pandas as pd
df=pd.DataFrame(bus_data)

df.to_csv('FINALFINAL.csv', index=False)

df.isnull().sum()
df.fillna(0, inplace=True)

#######################SQL#########################
import pymysql
myconnection = pymysql.connect(host='127.0.0.1',user='root',password='root')
myconnection.cursor().execute("create database redbus")

a=",".join(f"{i} {j}"
for i,j in zip(df.columns,df.dtypes)).replace("float64","float").replace("category","text").replace("int64","int").replace("object","text")

f"create table bus ({a})"

myconnection.cursor().execute(f"create table redbus.bus ({a})")

len(df)

sql = "insert into redbus.bus values"
for i in range(len(df)):
    myconnection.cursor().execute(f"{sql} {tuple(df.iloc[i])}")
    myconnection.commit()

df


In [ ]:
################################################STREAMLIT#############################################

In [ ]:
import streamlit as st
import pandas as pd
import pymysql
from datetime import time

st.image("C:/Users/Lenovo/Desktop/redbus2.png")
st.title(":red[RED BUS]")

mydb = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='root',
    database='redbus'
)

mycursor = mydb.cursor()


mycursor.execute('SELECT DISTINCT route_name FROM bus')
route_names = mycursor.fetchall()
route_name_options = [route[0] for route in route_names]

def filter_route_name(search_term):
    if search_term == "":
        return route_name_options
    return [option for option in route_name_options if search_term.lower() in option.lower()]

search_term = st.text_input('Search:', '')

filtered_route_name = filter_route_name(search_term)


if filtered_route_name:
    selected_option = st.selectbox('Select an option:', filtered_route_name)
else:
    st.write('No options found.')

################## BUS_TYPE

query_2 = '''
SELECT DISTINCT
    bus_type,
    CASE
        WHEN bus_type LIKE '%NON AC%'
             OR bus_type LIKE '%Non A/C%'
             OR bus_type LIKE '%NON A/C%'
             OR bus_type LIKE '%Non AC%'
             OR LOWER(bus_type) LIKE '%non%'
             OR LOWER(bus_type) LIKE '%non a/c%'
             OR LOWER(bus_type) LIKE '%non ac%'  THEN 'Non A/C'
        WHEN bus_type LIKE '%A/C%'
             OR bus_type LIKE '%AC%'
             OR bus_type LIKE '%a/c%'
             OR bus_type LIKE '%ac%' THEN 'A/C'
        ELSE 'None of these'
    END AS category
FROM bus
ORDER BY category
'''

mycursor.execute(query_2)
#fetch2 = select * from bus
fetch2 = mycursor.fetchall()
df_bus_type = pd.DataFrame(fetch2, columns=['bus_type', 'category'])
bus_type_list = df_bus_type['category'].unique().tolist()
bus_type_list.insert(0, "Choose your bus_type")
bus_type = st.selectbox(':red[Select Your Bus Type:]', bus_type_list)

####### DEPARTURE TIME ############

query_3 = '''SELECT DISTINCT departure_time FROM bus'''
mycursor.execute(query_3)
fetch3 = mycursor.fetchall()
df_departure_time = pd.DataFrame(fetch3, columns=['departure_time'])
time_format = '%H:%M'
df_departure_time['departure_time'] = pd.to_datetime(df_departure_time['departure_time'], format=time_format, errors='coerce')
departure_times = [time_obj.time() for time_obj in df_departure_time['departure_time'].dropna()]
default_time = departure_times[0] if departure_times else time(0, 0)
departure_time = st.time_input(':red[Select Your Departure Time :]', default_time)

###############  ARRIVAL TIME ######################
query_4 = '''SELECT DISTINCT reaching_time FROM bus'''
mycursor.execute(query_4)
fetch4 = mycursor.fetchall()
df_reaching_time = pd.DataFrame(fetch4, columns=['reaching_time'])
time_format = '%H:%M'
df_reaching_time['reaching_time'] = pd.to_datetime(df_reaching_time['reaching_time'], format=time_format, errors='coerce')
reaching_times = [time_obj.time() for time_obj in df_reaching_time['reaching_time'].dropna()]
reaching_time = st.time_input(':red[Select Your Reaching Time :]', default_time)


########## STAR RATING

query_5 = '''
SELECT DISTINCT star_rating
FROM bus
ORDER BY star_rating DESC '''
mycursor.execute(query_5)
fetch5 = mycursor.fetchall()
df_star_rating = pd.DataFrame(fetch5, columns=['star_rating'])
min_star_rating = int(df_star_rating['star_rating'].min())
max_star_rating = int(df_star_rating['star_rating'].max())
star_rating_range = st.slider(
    ':red[Select Your Star Rating Range:]',
    min_value=min_star_rating,
    max_value=max_star_rating,
    value=(min_star_rating, max_star_rating),
    step=1
)
min_star_rating, max_star_rating = star_rating_range

########## PRICE

query_6 = '''SELECT MIN(CAST(REPLACE(REPLACE(price, 'INR ', ''), ',', '') AS DECIMAL(10, 2))) AS min_price,
MAX(CAST(REPLACE(REPLACE(price, 'INR ', ''), ',', '') AS DECIMAL(10, 2))) AS max_price FROM bus'''
mycursor.execute(query_6)
fetch6 = mycursor.fetchall()
min_price = fetch6[0][0] if fetch6[0][0] is not None else 0
max_price = fetch6[0][1] if fetch6[0][1] is not None else 0
price_range = st.slider(':red[Select Price Range:]', min_value=float(min_price), max_value=float(max_price), value=(float(min_price), float(max_price)), step=0.1)

# Submit button
if st.button('Submit'):

    st.write(f'You selected Route: {selected_option}')
    st.write(f'Bus Type: {bus_type}')
    st.write(f'Departure Time: {departure_time.strftime("%H:%M")}')
    st.write(f'Reaching Time: {reaching_time.strftime("%H:%M")}')
    st.write(f'Star Rating: {star_rating_range}')
    st.write(f'Price Range: {price_range}')


    # Construct and execute the SQL query
    query = f'''
    SELECT * FROM bus
    WHERE route_name = '{selected_option}'
    AND LOWER('{bus_type}') LIKE LOWER(CONCAT('%', '{bus_type}', '%'))
    AND star_rating BETWEEN {min_star_rating} AND {max_star_rating}
    AND CAST(REPLACE(REPLACE(price, 'INR ', ''), ',', '') AS DECIMAL(10, 2)) BETWEEN {price_range[0]} AND {price_range[1]}
    AND departure_time = '{departure_time.strftime("%H:%M")}'
    AND reaching_time = '{reaching_time.strftime("%H:%M")}' '''

    try:
        mycursor.execute(query)
        filtered_data = pd.DataFrame(mycursor.fetchall(), columns=[i[0] for i in mycursor.description])
        if filtered_data.empty:
            st.write('No results found.')
        else:
            st.write('Filtered Results:')
            st.dataframe(filtered_data)
    except Exception as e:
        st.error(f"An error occurred: {e}")


mydb.close()
